In [42]:
from nozzle.client import Client
from nozzle.util import to_hex
import os.path
import time
from nozzle.util import Abi

import time
import pandas as pd

client_url = "grpc+tcp://34.122.177.97:80"
client = Client(client_url)

In [43]:
def elapsed(start):
    return round(time.time() - start, 4)
    
    

In [49]:
def process_query(client, query):
    # df = pd.DataFrame()

    print(query)
    start = time.time()

    result_stream = client.get_sql(query)
    print('time to establish stream: ', elapsed(start), 's')

    total_events = 0
    try:
        batch = next(result_stream)
        print('time for first batch ', elapsed(start), 's')
        total_events += batch.num_rows
        df = batch.to_pandas().map(to_hex)

        print('The type of df is ', type(df))
        
        batch_start = time.time()

        for batch in result_stream:
            total_events += batch.num_rows
            print('received batch of ', batch.num_rows, ' rows in ', elapsed(batch_start), 's')
            batch_start = time.time()
            new_df = batch.to_pandas().map(to_hex)
            # Concatenate the df dataframe to the previous dataframe
            df = pd.concat([df, new_df])

            

        print('total rows: ', total_events)
        print('total time to consume the stream: ', elapsed(start), 's')
        print('rows/s: ', total_events / elapsed(start))
        return df 

    except StopIteration:
        print("No more batches available in the result stream.")

In [50]:
# The address of Staking Contract
StakingExtension_address = 'F55041E37E12cD407ad00CE2910B8269B01263b9'
StakingExtension_path =  "abis/StakingExtension.json"


IStakingStitched_address = 'F55041E37E12cD407ad00CE2910B8269B01263b9'
IStakingStitched_path = "abis/IStakingStitched.json"

In [80]:
# address of arbitrum
ServiceRegistry_address = '072884c745c0A23144753335776c99BE22588f8A' 


ServiceRegistry_path =  "abis/ServiceRegistry.json"
ServiceRegistryAbi = Abi(ServiceRegistry_path)

ServiceRegistered_sig = ServiceRegistryAbi.events["ServiceRegistered"].signature()
ServiceRegistered_query = f"""
    select  pc.block_num as block_number,
            pc.ServiceRegistered['indexer'] as indexer,
            pc.ServiceRegistered['url'] as url,
            pc.ServiceRegistered['geohash'] as geohash
      from (select l.block_num,
                   evm_decode(l.topic1, l.topic2, l.topic3, l.data, '{ServiceRegistered_sig}') as ServiceRegistered
                from logs l
             where l.address = arrow_cast(x'{ServiceRegistry_address}', 'FixedSizeBinary(20)')
               and l.topic0 = evm_topic('{ServiceRegistered_sig}')
) pc
"""
ServiceRegistered = process_query(client, ServiceRegistered_query)



    select  pc.block_num as block_number,
            pc.ServiceRegistered['indexer'] as indexer,
            pc.ServiceRegistered['url'] as url,
            pc.ServiceRegistered['geohash'] as geohash
      from (select l.block_num,
                   evm_decode(l.topic1, l.topic2, l.topic3, l.data, 'ServiceRegistered(address indexed indexer,string url,string geohash)') as ServiceRegistered
                from logs l
             where l.address = arrow_cast(x'072884c745c0A23144753335776c99BE22588f8A', 'FixedSizeBinary(20)')
               and l.topic0 = evm_topic('ServiceRegistered(address indexed indexer,string url,string geohash)')
) pc

time to establish stream:  0.1535 s
No more batches available in the result stream.


In [60]:
ServiceRegistry_address = 'D0C9DaCf1e515615b0581c8D7E295E296Ec26E6'

ServiceRegistry_path =  "abis/ServiceRegistry.json"
ServiceRegistryAbi = Abi(ServiceRegistry_path)

ServiceRegistered_sig = ServiceRegistryAbi.events["ServiceRegistered"].signature()
ServiceRegistered_query = f"""
    select  pc.block_num as block_number,
            pc.ServiceRegistered['indexer'] as indexer,
            pc.ServiceRegistered['url'] as url,
            pc.ServiceRegistered['geohash'] as geohash
      from (select l.block_num,
                   evm_decode(l.topic1, l.topic2, l.topic3, l.data, '{ServiceRegistered_sig}') as ServiceRegistered
                from logs l
             where l.address = arrow_cast(x'{ServiceRegistry_address}', 'FixedSizeBinary(20)')
               and l.topic0 = evm_topic('{ServiceRegistered_sig}')
) pc
"""
ServiceRegistered = process_query(client, ServiceRegistered_query)


StakingExtensionAbi = Abi(StakingExtension_path)

StakingExtension_created_sig_1 = StakingExtensionAbi.events["StakeDelegated"].signature()
StakingExtension_created_sig_2 = StakingExtensionAbi.events["StakeDelegatedLocked"].signature()


IStakingStitchedAbi = Abi(IStakingStitched_path)

AllocationCreated_sig = IStakingStitchedAbi.events["AllocationCreated"].signature()

AllocationCreated_query = f"""
    select  pc.block_num as block_number,
            pc.AllocationCreated['metadata'] as metadata,
            pc.AllocationCreated['allocationID'] as allocationID,
            pc.AllocationCreated['tokens'] as tokens,
            pc.AllocationCreated['epoch'] as epoch,
            pc.AllocationCreated['subgraphDeploymentID'] as subgraphDeploymentID,
            pc.AllocationCreated['indexer'] as indexer,
            pc.AllocationCreated['rebateFees'] as rebateFees
      from (select l.block_num,
                   evm_decode(l.topic1, l.topic2, l.topic3, l.data, '{AllocationCreated_sig}') as AllocationCreated
                from logs l
             where l.address = arrow_cast(x'{IStakingStitched_address}', 'FixedSizeBinary(20)')
               and l.topic0 = evm_topic('{AllocationCreated_sig}')
) pc
"""

AllocationCreated = process_query(client, AllocationCreated_query)

AllocationCollected_sig = IStakingStitchedAbi.events["AllocationCollected"].signature()

AllocationCollected_query = f"""
    SELECT  pc.block_num AS block_number,
            pc.AllocationCollected['indexer'] AS indexer,
            pc.AllocationCollected['subgraphDeploymentID'] AS subgraphDeploymentID,
            pc.AllocationCollected['epoch'] AS epoch,
            pc.AllocationCollected['tokens'] AS tokens,
            pc.AllocationCollected['allocationID'] AS allocationID,
            pc.AllocationCollected['from'] AS from,
            pc.AllocationCollected['curationFees'] AS curationFees,
            pc.AllocationCollected['rebateFees'] AS rebateFees
      FROM (SELECT l.block_num,
                   evm_decode(l.topic1, l.topic2, l.topic3, l.data, '{AllocationCollected_sig}') AS AllocationCollected
              FROM logs l
             WHERE l.address = arrow_cast(x'{IStakingStitched_address}', 'FixedSizeBinary(20)')
               AND l.topic0 = evm_topic('{AllocationCollected_sig}')
) pc
"""

AllocationCollected = process_query(client, AllocationCollected_query)



StakeDelegated_sig = IStakingStitchedAbi.events["StakeDelegated"].signature()

StakeDelegated_query = f"""
    SELECT  pc.block_num AS block_number,
            pc.StakeDelegated['indexer'] AS indexer,
            pc.StakeDelegated['delegator'] AS delegator,
            pc.StakeDelegated['tokens'] AS tokens,
            pc.StakeDelegated['shares'] AS shares
      FROM (SELECT l.block_num,
                   evm_decode(l.topic1, l.topic2, l.topic3, l.data, '{StakeDelegated_sig}') AS StakeDelegated
              FROM logs l
             WHERE l.address = arrow_cast(x'{IStakingStitched_address}', 'FixedSizeBinary(20)')
               AND l.topic0 = evm_topic('{StakeDelegated_sig}')
) pc
"""

StakeDelegated = process_query(client, StakeDelegated_query)

StakeDeposited_sig = IStakingStitchedAbi.events["StakeDeposited"].signature()

StakeDeposited_query = f"""
    SELECT  pc.block_num AS block_number,
            pc.StakeDeposited['indexer'] AS indexer,
            pc.StakeDeposited['tokens'] AS tokens
      FROM (SELECT l.block_num,
                   evm_decode(l.topic1, l.topic2, l.topic3, l.data, '{StakeDeposited_sig}') AS StakeDeposited
              FROM logs l
             WHERE l.address = arrow_cast(x'{IStakingStitched_address}', 'FixedSizeBinary(20)')
               AND l.topic0 = evm_topic('{StakeDeposited_sig}')
) pc
"""

StakeDeposited = process_query(client, StakeDeposited_query)

StakeLocked_sig = IStakingStitchedAbi.events["StakeLocked"].signature()

StakeLocked_query = f"""
    SELECT  pc.block_num AS block_number,
            pc.StakeLocked['indexer'] AS indexer,
            pc.StakeLocked['tokens'] AS tokens,
            pc.StakeLocked['until'] AS until
      FROM (SELECT l.block_num,
                   evm_decode(l.topic1, l.topic2, l.topic3, l.data, '{StakeLocked_sig}') AS StakeLocked
              FROM logs l
             WHERE l.address = arrow_cast(x'{IStakingStitched_address}', 'FixedSizeBinary(20)')
               AND l.topic0 = evm_topic('{StakeLocked_sig}')
) pc
"""

StakeLocked = process_query(client, StakeLocked_query)

RebateClaimed_sig = IStakingStitchedAbi.events["RebateClaimed"].signature()

RebateClaimed_query = f"""
    SELECT  pc.block_num AS block_number,
            pc.RebateClaimed['indexer'] AS indexer,
            pc.RebateClaimed['subgraphDeploymentID'] AS subgraphDeploymentID,
            pc.RebateClaimed['allocationID'] AS allocationID,
            pc.RebateClaimed['epoch'] AS epoch,
            pc.RebateClaimed['forEpoch'] AS forEpoch,
            pc.RebateClaimed['tokens'] AS tokens,
            pc.RebateClaimed['unclaimedAllocationsCount'] AS unclaimedAllocationsCount,
            pc.RebateClaimed['delegationFees'] AS delegationFees
      FROM (SELECT l.block_num,
                   evm_decode(l.topic1, l.topic2, l.topic3, l.data, '{RebateClaimed_sig}') AS RebateClaimed
              FROM logs l
             WHERE l.address = arrow_cast(x'{IStakingStitched_address}', 'FixedSizeBinary(20)')
               AND l.topic0 = evm_topic('{RebateClaimed_sig}')
) pc
"""

RebateClaimed = process_query(client, RebateClaimed_query)

StakeDelegatedLocked_sig = IStakingStitchedAbi.events["StakeDelegatedLocked"].signature()

StakeDelegatedLocked_query = f"""
    SELECT  pc.block_num AS block_number,
            pc.StakeDelegatedLocked['indexer'] AS indexer,
            pc.StakeDelegatedLocked['delegator'] AS delegator,
            pc.StakeDelegatedLocked['tokens'] AS tokens,
            pc.StakeDelegatedLocked['shares'] AS shares,
            pc.StakeDelegatedLocked['until'] AS until
      FROM (SELECT l.block_num,
                   evm_decode(l.topic1, l.topic2, l.topic3, l.data, '{StakeDelegatedLocked_sig}') AS StakeDelegatedLocked
              FROM logs l
             WHERE l.address = arrow_cast(x'{IStakingStitched_address}', 'FixedSizeBinary(20)')
               AND l.topic0 = evm_topic('{StakeDelegatedLocked_sig}')
) pc
"""

StakeDelegatedLocked = process_query(client, StakeDelegatedLocked_query)

RebateCollected_sig = IStakingStitchedAbi.events["RebateCollected"].signature()

RebateCollected_query = f"""
    SELECT  pc.block_num AS block_number,
            pc.RebateCollected['assetHolder'] AS assetHolder,
            pc.RebateCollected['indexer'] AS indexer,
            pc.RebateCollected['subgraphDeploymentID'] AS subgraphDeploymentID,
            pc.RebateCollected['allocationID'] AS allocationID,
            pc.RebateCollected['epoch'] AS epoch,
            pc.RebateCollected['tokens'] AS tokens,
            pc.RebateCollected['protocolTax'] AS protocolTax,
            pc.RebateCollected['curationFees'] AS curationFees,
            pc.RebateCollected['queryFees'] AS queryFees,
            pc.RebateCollected['queryRebates'] AS queryRebates,
            pc.RebateCollected['delegationRewards'] AS delegationRewards
      FROM (SELECT l.block_num,
                   evm_decode(l.topic1, l.topic2, l.topic3, l.data, '{RebateCollected_sig}') AS RebateCollected
              FROM logs l
             WHERE l.address = arrow_cast(x'{IStakingStitched_address}', 'FixedSizeBinary(20)')
               AND l.topic0 = evm_topic('{RebateCollected_sig}')
) pc
"""

RebateCollected = process_query(client, RebateCollected_query)

StakeWithdrawn_sig = IStakingStitchedAbi.events["StakeWithdrawn"].signature()

StakeWithdrawn_query = f"""
    SELECT  pc.block_num AS block_number,
            pc.StakeWithdrawn['indexer'] AS indexer,
            pc.StakeWithdrawn['tokens'] AS tokens
      FROM (SELECT l.block_num,
                   evm_decode(l.topic1, l.topic2, l.topic3, l.data, '{StakeWithdrawn_sig}') AS StakeWithdrawn
              FROM logs l
             WHERE l.address = arrow_cast(x'{IStakingStitched_address}', 'FixedSizeBinary(20)')
               AND l.topic0 = evm_topic('{StakeWithdrawn_sig}')
) pc
"""

StakeWithdrawn = process_query(client, StakeWithdrawn_query)

StakeSlashed_sig = IStakingStitchedAbi.events["StakeSlashed"].signature()

StakeSlashed_query = f"""
    SELECT  pc.block_num AS block_number,
            pc.StakeSlashed['indexer'] AS indexer,
            pc.StakeSlashed['tokens'] AS tokens,
            pc.StakeSlashed['reward'] AS reward,
            pc.StakeSlashed['beneficiary'] AS beneficiary
      FROM (SELECT l.block_num,
                   evm_decode(l.topic1, l.topic2, l.topic3, l.data, '{StakeSlashed_sig}') AS StakeSlashed
              FROM logs l
             WHERE l.address = arrow_cast(x'{IStakingStitched_address}', 'FixedSizeBinary(20)')
               AND l.topic0 = evm_topic('{StakeSlashed_sig}')
) pc
"""

StakeSlashed = process_query(client, StakeSlashed_query)




    select  pc.block_num as block_number,
            pc.ServiceRegistered['url'] as url,
            pc.ServiceRegistered['geohash'] as geohash
      from (select l.block_num,
                   evm_decode(l.topic1, l.topic2, l.topic3, l.data, 'ServiceRegistered(address indexed indexer,string url,string geohash)') as ServiceRegistered
                from logs l
             where l.address = arrow_cast(x'D0C9DaCf1e515615b0581c8D7E295E296Ec26E6', 'FixedSizeBinary(20)')
               and l.topic0 = evm_topic('ServiceRegistered(address indexed indexer,string url,string geohash)')
) pc

time to establish stream:  0.1491 s
No more batches available in the result stream.

    select  pc.block_num as block_number,
            pc.AllocationCreated['metadata'] as metadata,
            pc.AllocationCreated['allocationID'] as allocationID,
            pc.AllocationCreated['tokens'] as tokens,
            pc.AllocationCreated['epoch'] as epoch,
            pc.AllocationCreated['subgraphDeploym

In [66]:

print("This table's columns are: ", StakeSlashed.columns.tolist())


This table's columns are:  ['block_number', 'indexer', 'tokens', 'reward', 'beneficiary']


In [68]:

print("This table's columns are: ", AllocationCreated.columns.tolist())


This table's columns are:  ['block_number', 'metadata', 'allocationid', 'tokens', 'epoch', 'subgraphdeploymentid', 'indexer']


In [73]:
L1Staking_address = 'F55041E37E12cD407ad00CE2910B8269B01263b9'
L1Staking_path =  "abis/L1Staking.json"
L1StakingAbi = Abi(L1Staking_path)

IndexerStakeTransferredToL2_sig = L1StakingAbi.events["IndexerStakeTransferredToL2"].signature()

IndexerStakeTransferredToL2_query = f"""
    SELECT  pc.block_num AS block_number,
            pc.IndexerStakeTransferredToL2['indexer'] AS indexer,
            pc.IndexerStakeTransferredToL2['l2Indexer'] AS l2Indexer,
            pc.IndexerStakeTransferredToL2['transferredStakeTokens'] AS transferredStakeTokens
      FROM (SELECT l.block_num,
                   evm_decode(l.topic1, l.topic2, l.topic3, l.data, '{IndexerStakeTransferredToL2_sig}') AS IndexerStakeTransferredToL2
              FROM logs l
             WHERE l.address = arrow_cast(x'{L1Staking_address}', 'FixedSizeBinary(20)')
               AND l.topic0 = evm_topic('{IndexerStakeTransferredToL2_sig}')
) pc
"""

IndexerStakeTransferredToL2 = process_query(client, IndexerStakeTransferredToL2_query)



    SELECT  pc.block_num AS block_number,
            pc.IndexerStakeTransferredToL2['indexer'] AS indexer,
            pc.IndexerStakeTransferredToL2['l2Indexer'] AS l2Indexer,
            pc.IndexerStakeTransferredToL2['transferredStakeTokens'] AS transferredStakeTokens
      FROM (SELECT l.block_num,
                   evm_decode(l.topic1, l.topic2, l.topic3, l.data, 'IndexerStakeTransferredToL2(address indexed indexer,address indexed l2Indexer,uint256 transferredStakeTokens)') AS IndexerStakeTransferredToL2
              FROM logs l
             WHERE l.address = arrow_cast(x'F55041E37E12cD407ad00CE2910B8269B01263b9', 'FixedSizeBinary(20)')
               AND l.topic0 = evm_topic('IndexerStakeTransferredToL2(address indexed indexer,address indexed l2Indexer,uint256 transferredStakeTokens)')
) pc

time to establish stream:  0.1583 s
time for first batch  88.7408 s
The type of df is  <class 'pandas.core.frame.DataFrame'>
received batch of  7  rows in  0.3904 s
received batch of  21 

In [74]:
DelegationTransferredToL2_sig = L1StakingAbi.events["DelegationTransferredToL2"].signature()
DelegationTransferredToL2_query = f"""
    SELECT  pc.block_num AS block_number,
            pc.DelegationTransferredToL2['delegator'] AS delegator,
            pc.DelegationTransferredToL2['l2Delegator'] AS l2Delegator,
            pc.DelegationTransferredToL2['indexer'] AS indexer,
            pc.DelegationTransferredToL2['l2Indexer'] AS l2Indexer,
            pc.DelegationTransferredToL2['transferredDelegationTokens'] AS transferredDelegationTokens
      FROM (SELECT l.block_num,
                   evm_decode(l.topic1, l.topic2, l.topic3, l.data, '{DelegationTransferredToL2_sig}') AS DelegationTransferredToL2
              FROM logs l
             WHERE l.address = arrow_cast(x'{L1Staking_address}', 'FixedSizeBinary(20)')
               AND l.topic0 = evm_topic('{DelegationTransferredToL2_sig}')
) pc
"""

DelegationTransferredToL2 = process_query(client, DelegationTransferredToL2_query)



    SELECT  pc.block_num AS block_number,
            pc.DelegationTransferredToL2['delegator'] AS delegator,
            pc.DelegationTransferredToL2['l2Delegator'] AS l2Delegator,
            pc.DelegationTransferredToL2['indexer'] AS indexer,
            pc.DelegationTransferredToL2['l2Indexer'] AS l2Indexer,
            pc.DelegationTransferredToL2['transferredDelegationTokens'] AS transferredDelegationTokens
      FROM (SELECT l.block_num,
                   evm_decode(l.topic1, l.topic2, l.topic3, l.data, 'DelegationTransferredToL2(address indexed delegator,address indexed l2Delegator,address indexed indexer,address l2Indexer,uint256 transferredDelegationTokens)') AS DelegationTransferredToL2
              FROM logs l
             WHERE l.address = arrow_cast(x'F55041E37E12cD407ad00CE2910B8269B01263b9', 'FixedSizeBinary(20)')
               AND l.topic0 = evm_topic('DelegationTransferredToL2(address indexed delegator,address indexed l2Delegator,address indexed indexer,address l2

In [77]:
RewardsManager_address = '9Ac758AB77733b4150A901ebd659cbF8cB93ED66'
RewardsManager_path =  "abis/RewardsManager.json"
RewardsManagerAbi = Abi(RewardsManager_path)
RewardsAssigned_sig = RewardsManagerAbi.events["RewardsAssigned"].signature()


RewardsAssigned_query = f"""
    SELECT  pc.block_num AS block_number,
            pc.RewardsAssigned['indexer'] AS indexer,
            pc.RewardsAssigned['allocationID'] AS allocationID,
            pc.RewardsAssigned['epoch'] AS epoch,
            pc.RewardsAssigned['amount'] AS amount
      FROM (SELECT l.block_num,
                   evm_decode(l.topic1, l.topic2, l.topic3, l.data, '{RewardsAssigned_sig}') AS RewardsAssigned
              FROM logs l
             WHERE l.address = arrow_cast(x'{RewardsManager_address}', 'FixedSizeBinary(20)')
               AND l.topic0 = evm_topic('{RewardsAssigned_sig}')
) pc
"""

RewardsAssigned = process_query(client, RewardsAssigned_query)



    SELECT  pc.block_num AS block_number,
            pc.RewardsAssigned['indexer'] AS indexer,
            pc.RewardsAssigned['allocationID'] AS allocationID,
            pc.RewardsAssigned['epoch'] AS epoch,
            pc.RewardsAssigned['amount'] AS amount
      FROM (SELECT l.block_num,
                   evm_decode(l.topic1, l.topic2, l.topic3, l.data, 'RewardsAssigned(address indexed indexer,address indexed allocationID,uint256 epoch,uint256 amount)') AS RewardsAssigned
              FROM logs l
             WHERE l.address = arrow_cast(x'9Ac758AB77733b4150A901ebd659cbF8cB93ED66', 'FixedSizeBinary(20)')
               AND l.topic0 = evm_topic('RewardsAssigned(address indexed indexer,address indexed allocationID,uint256 epoch,uint256 amount)')
) pc

time to establish stream:  0.1174 s
time for first batch  74.9524 s
The type of df is  <class 'pandas.core.frame.DataFrame'>
received batch of  1208  rows in  4.0364 s
received batch of  653  rows in  0.4731 s
received batch of  2080

To process each event in the block number order using the provided Typescript logic, we will need to:

Combine all event DataFrames into one DataFrame.
Sort the combined DataFrame by block_number.
Process each event based on its type usming the provided logic.


In [81]:

# Adding the event_type column to each DataFrame
# ServiceRegistered = ServiceRegistered.assign(event_type='ServiceRegistered')
AllocationCreated = AllocationCreated.assign(event_type='AllocationCreated')
AllocationCollected = AllocationCollected.assign(event_type='AllocationCollected')
IndexerStakeTransferredToL2 = IndexerStakeTransferredToL2.assign(event_type='IndexerStakeTransferredToL2')

StakeDelegated = StakeDelegated.assign(event_type='StakeDelegated')
StakeDeposited = StakeDeposited.assign(event_type='StakeDeposited')
StakeLocked = StakeLocked.assign(event_type='StakeLocked')
RebateClaimed = RebateClaimed.assign(event_type='RebateClaimed')
DelegationTransferredToL2 = DelegationTransferredToL2.assign(event_type='DelegationTransferredToL2')

StakeDelegatedLocked = StakeDelegatedLocked.assign(event_type='StakeDelegatedLocked')
RewardsAssigned = RewardsAssigned.assign(event_type='RewardsAssigned')
RebateCollected = RebateCollected.assign(event_type='RebateCollected')
StakeWithdrawn = StakeWithdrawn.assign(event_type='StakeWithdrawn')
StakeSlashed = StakeSlashed.assign(event_type='StakeSlashed')

# Combine all DataFrames
all_events_df = pd.concat([
    ServiceRegistered,
    AllocationCreated,
    AllocationCollected,
    IndexerStakeTransferredToL2,
    StakeDelegated,
    StakeDeposited,
    StakeLocked,
    RebateClaimed,
    DelegationTransferredToL2,
    StakeDelegatedLocked,
    RewardsAssigned,
    RebateCollected,
    StakeWithdrawn,
    StakeSlashed
])

# Sort by block_number
all_events_df = all_events_df.sort_values(by='block_number').reset_index(drop=True)


In [91]:
from collections import defaultdict

# Initialize indexerStored
indexerStored = defaultdict(lambda: {
    'stakedTokens': 0,
    'geoHash': '',
    'allocatedTokens': 0,
    'totalAllocationCount': 0,
    'allocationCount': 0,
    'queryFeesCollected': 0,
    'delegatedTokens': 0,
    'lockedTokens': 0,
    'queryFeeRebates': 0,
    'indexingRewardCut': 0,
    'tokensLockedstakedTokens': 0,
    'indexerIndexingRewards': 0
})

# Function to process events
def process_events(df, indexerStored):
    for _, row in df.iterrows():
        event_type = row['event_type']
        indexer = row['indexer']
        
        if event_type == 'ServiceRegistered':
            indexerStored[indexer]['stakedTokens'] -= row['transferredStakeTokens']
            indexerStored[indexer]['geoHash'] = row['geoHash']
        
        elif event_type == 'AllocationCreated':
            indexerStored[indexer]['allocatedTokens'] += row['tokens']
            indexerStored[indexer]['totalAllocationCount'] += 1
            indexerStored[indexer]['allocationCount'] += 1
            # indexerStored[indexer]['queryFeesCollected'] += row['rebateFees']
        
        elif event_type == 'AllocationCollected':
            indexerStored[indexer]['queryFeesCollected'] += 0

            # indexerStored[indexer]['queryFeesCollected'] += row['rebateFees']
        
        elif event_type == 'IndexerStakeTransferredToL2':
            continue
            # indexerStored[indexer]['stakedTokens'] -= row['transferredStakeTokens']
        
        elif event_type == 'StakeDelegated':
            indexerStored[indexer]['delegatedTokens'] += row['tokens']
        
        elif event_type == 'StakeDeposited':
            indexerStored[indexer]['stakedTokens'] += row['tokens']
        
        elif event_type == 'StakeLocked':
            indexerStored[indexer]['lockedTokens'] = row['tokens']
        
        elif event_type == 'RebateClaimed':
            # indexerStored[indexer]['delegatedTokens'] += row['delegationFees']
            indexerStored[indexer]['queryFeeRebates'] += row['tokens']
        
        elif event_type == 'DelegationTransferredToL2':
            # indexerStored[indexer]['delegatedTokens'] -= row['transferredDelegationTokens']
            continue
        
        elif event_type == 'StakeDelegatedLocked':
            indexerStored[indexer]['delegatedTokens'] -= row['tokens']
        
        elif event_type == 'RewardsAssigned':
            if indexerStored[indexer]['delegatedTokens'] == 0:
                indexerStored[indexer]['indexerIndexingRewards'] = row['amount']
            else:
                indexerStored[indexer]['indexerIndexingRewards'] = row['amount'] * indexerStored[indexer]['indexingRewardCut'] // 1000000
            indexerStored[indexer]['delegatedTokens'] += indexerStored[indexer]['indexerIndexingRewards']
        
        elif event_type == 'RebateCollected':
            continue
            # indexerStored[indexer]['delegatedTokens'] += row['delegationRewards']
            # indexerStored[indexer]['queryFeesCollected'] += row['queryFees']
            # indexerStored[indexer]['queryFeeRebates'] += row['queryRebates']
        
        elif event_type == 'StakeWithdrawn':
            indexerStored[indexer]['lockedTokens'] -= row['tokens']
        
        elif event_type == 'StakeSlashed':
            indexerStored[indexer]['lockedTokens'] = indexerStored[indexer]['tokensLockedstakedTokens']
            indexerStored[indexer]['stakedTokens'] -= row['tokens']

# Process all events
process_events(all_events_df, indexerStored)

# Print the updated indexerStored
print(indexerStored)


defaultdict(<function <lambda> at 0x14221d800>, {'0x06590a641dc3eb43f2cebe435576389f209116da': {'stakedTokens': Decimal('10000000000000000000'), 'geoHash': '', 'allocatedTokens': 0, 'totalAllocationCount': 0, 'allocationCount': 0, 'queryFeesCollected': 0, 'delegatedTokens': 0, 'lockedTokens': Decimal('0'), 'queryFeeRebates': 0, 'indexingRewardCut': 0, 'tokensLockedstakedTokens': 0, 'indexerIndexingRewards': 0}, '0xf62de2a9d8b8033a5b66f497dca1bd91080316fb': {'stakedTokens': Decimal('100000000000000000000000'), 'geoHash': '', 'allocatedTokens': 0, 'totalAllocationCount': 0, 'allocationCount': 0, 'queryFeesCollected': 0, 'delegatedTokens': Decimal('0'), 'lockedTokens': Decimal('0'), 'queryFeeRebates': 0, 'indexingRewardCut': 0, 'tokensLockedstakedTokens': 0, 'indexerIndexingRewards': 0}, '0x62a0bd1d110ff4e5b793119e95fc07c9d1fc8c4a': {'stakedTokens': Decimal('4785367456361618270620872'), 'geoHash': '', 'allocatedTokens': Decimal('3653311921070000000000000000'), 'totalAllocationCount': 1574

In [92]:
indexerStored

defaultdict(<function __main__.<lambda>()>,
            {'0x06590a641dc3eb43f2cebe435576389f209116da': {'stakedTokens': Decimal('10000000000000000000'),
              'geoHash': '',
              'allocatedTokens': 0,
              'totalAllocationCount': 0,
              'allocationCount': 0,
              'queryFeesCollected': 0,
              'delegatedTokens': 0,
              'lockedTokens': Decimal('0'),
              'queryFeeRebates': 0,
              'indexingRewardCut': 0,
              'tokensLockedstakedTokens': 0,
              'indexerIndexingRewards': 0},
             '0xf62de2a9d8b8033a5b66f497dca1bd91080316fb': {'stakedTokens': Decimal('100000000000000000000000'),
              'geoHash': '',
              'allocatedTokens': 0,
              'totalAllocationCount': 0,
              'allocationCount': 0,
              'queryFeesCollected': 0,
              'delegatedTokens': Decimal('0'),
              'lockedTokens': Decimal('0'),
              'queryFeeRebates': 0,
  

In [99]:

# Convert the defaultdict to a regular dictionary
indexer_dict = {key: dict(value) for key, value in indexerStored.items()}

# Convert the dictionary to a pandas DataFrame
indexer_df = pd.DataFrame.from_dict(indexer_dict, orient='index')

# Reset index to turn the index into a column
indexer_df.reset_index(inplace=True)
indexer_df.rename(columns={'index': 'indexer'}, inplace=True)

# Convert large numeric columns to string to avoid numeric range issues in Parquet
columns_to_convert = ['stakedTokens', 'allocatedTokens', 'queryFeesCollected', 'delegatedTokens', 'lockedTokens', 'queryFeeRebates', 'tokensLockedstakedTokens', 'indexerIndexingRewards']
indexer_df[columns_to_convert] = indexer_df[columns_to_convert].applymap(str)


# Save the DataFrame to a Parquet file
indexer_df.to_parquet('indexerStored.parquet', engine='pyarrow')

# Print the DataFrame to check the conversion
print(indexer_df)

                                        indexer               stakedTokens  \
0    0x06590a641dc3eb43f2cebe435576389f209116da       10000000000000000000   
1    0xf62de2a9d8b8033a5b66f497dca1bd91080316fb   100000000000000000000000   
2    0x62a0bd1d110ff4e5b793119e95fc07c9d1fc8c4a  4785367456361618270620872   
3    0xf3f182e859f789af8d6c2223d15c691471c79a2b  2331563819532313429219378   
4    0x163a2ebf98ef202da5834a33f7ce98098a0b7874   192308000000000000000000   
..                                          ...                        ...   
573  0xe2571c87f1433ea06be389e427af2a17bfd37fc0   210571267754876523306445   
574  0x9ada64ca2fa91318e2de0ff678a8a935859954f4   100674499853717515400000   
575  0x56598b0816dd39fea9aef8e4dbb49e87f5b10fbf   100000000000000000000000   
576  0x2d2771e17fe8daff29b6d903559ec1c8192945b0   100501767136255316115456   
577  0xd11eb5db7cfbb9ecae4b62e71ec0a461f6baf669   100000000000000000000000   

    geoHash               allocatedTokens  totalAllocationCount

/var/folders/5t/tktsv9cs629bjbjkqylhtw100000gn/T/ipykernel_97758/1637334497.py:13: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  indexer_df[columns_to_convert] = indexer_df[columns_to_convert].applymap(str)


In [97]:
from google.cloud import storage, bigquery


def upload_to_gcs(bucket_name, source_file_name, destination_blob_name):
    """Uploads a file to the bucket."""
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)

    blob.upload_from_filename(source_file_name)

    print(f"File {source_file_name} uploaded to {destination_blob_name}.")

# Example usage
bucket_name = 'nozzle-data-science'
source_file_name = '/Users/vivianpeng/nozzle/project-nozzle/python/examples/the_graph/pq_data/indexerStored.parquet'
destination_blob_name = 'path/in/bucket/indexer.parquet'

upload_to_gcs(bucket_name, source_file_name, destination_blob_name)

def load_parquet_to_bigquery(dataset_id, table_id, gcs_uri):
    """Loads a Parquet file from GCS into BigQuery."""
    client = bigquery.Client()

    table_ref = client.dataset(dataset_id).table(table_id)
    job_config = bigquery.LoadJobConfig(
        source_format=bigquery.SourceFormat.PARQUET,
    )

    load_job = client.load_table_from_uri(
        gcs_uri,
        table_ref,
        job_config=job_config
    )

    print(f"Starting job {load_job.job_id}")

    load_job.result()  # Waits for the job to complete.

    print(f"Job finished. Loaded {load_job.output_rows} rows into {dataset_id}:{table_id}")



/Users/vivianpeng/nozzle/project-nozzle/python/.venv/lib/python3.12/site-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
/Users/vivianpeng/nozzle/project-nozzle/python/.venv/lib/python3.12/site-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


File /Users/vivianpeng/nozzle/project-nozzle/python/examples/the_graph/pq_data/indexerStored.parquet uploaded to path/in/bucket/indexer.parquet.


In [100]:
# Example usage
dataset_id = 'internal_metrics'
table_id = 'indexer'
gcs_uri = f'gs://{bucket_name}/{destination_blob_name}'

load_parquet_to_bigquery(dataset_id, table_id, gcs_uri)


/Users/vivianpeng/nozzle/project-nozzle/python/.venv/lib/python3.12/site-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
/Users/vivianpeng/nozzle/project-nozzle/python/.venv/lib/python3.12/site-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


Starting job 84c531a7-0385-4c2e-868d-2fa7605a0727


BadRequest: 400 Error while reading data, error message: The value for column 'allocatedTokens' is out of valid NUMERIC range: Value is out of range after scaling to NUMERIC type; input length: 13; scale: 0; input data: 01a46d77c6247507abbfaff200. Consider specifying decimal target types to allow a larger range of values. See https://cloud.google.com/bigquery/docs/loading-data-cloud-storage-parquet#decimal_logical_type; in column 'allocatedTokens' File: gs://nozzle-data-science/path/in/bucket/indexer.parquet; reason: invalid, location: gs://nozzle-data-science/path/in/bucket/indexer.parquet, message: Error while reading data, error message: The value for column 'allocatedTokens' is out of valid NUMERIC range: Value is out of range after scaling to NUMERIC type; input length: 13; scale: 0; input data: 01a46d77c6247507abbfaff200. Consider specifying decimal target types to allow a larger range of values. See https://cloud.google.com/bigquery/docs/loading-data-cloud-storage-parquet#decima

In [24]:
!pip install google-cloud-storage google-cloud-bigquery


  Using cached google_auth-2.29.0-py2.py3-none-any.whl.metadata (4.7 kB)
  Using cached google_cloud_core-2.4.1-py2.py3-none-any.whl.metadata (2.7 kB)
  Using cached google_resumable_media-2.7.0-py2.py3-none-any.whl.metadata (2.2 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached protobuf-4.25.3-cp37-abi3-macosx_10_9_universal2.whl.metadata (541 bytes)
  Using cached proto_plus-1.23.0-py3-none-any.whl.metadata (2.2 kB)
  Using cached cachetools-5.3.3-py3-none-any.whl.metadata (5.3 kB)
  Using cached pyasn1_modules-0.4.0-py3-none-any.whl.metadata (3.4 kB)
  Using cached rsa-4.9-py3-none-any.whl.metadata (4.2 kB)
INFO: pip is looking at multiple versions of grpcio-status to determine which version is compatible with other requirements. This could take a while.
  Using cached grpcio_status-1.62.2-py3-none-any.whl.metadata (1.3 kB)
  Using cached pyasn1-0.6.0-py2.py3-none-any.whl.metadata

In [25]:
!pip list | grep google-cloud


google-cloud              0.34.0
google-cloud-bigquery     3.23.1
google-cloud-core         2.4.1
google-cloud-storage      2.16.0

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
